# Crypto / Squares vs. Cubes

Basically sending x0 back is sufficient, you get these values mod N (where f is the flag):
$$
\begin{align*}
m_0 &= p^2 + (q + rf)^2 \\
m_1 &= p^3 + (q + rf)^3 + (x_0 - x_1)^d
\end{align*}
$$

Rearrange and power by $e$ to get
$$x_0 - x_1 = \left(m_1 - p^3 - (q + rf)^3\right)^e$$

Now, consider $(q + rf)$ as a single variable, let's write it as $\zeta$ for clarity. Then we have these two equations:
$$
\begin{align*}
\zeta^2 - m_0 &\equiv 0 \pmod{p}\\
x_0 - x_1 + \left(\zeta^3 - m_1\right)^e &\equiv 0 \pmod{p}
\end{align*}
$$

Both are polynomials with $\zeta - (q+rf)$ a root, so we can just gcd them to learn it!

The only issue is we can't do mod p because we don't have p, but on the other hand it means that gcd-ing them mod N gives a multiple of p.

The other trick here is you can't really gcd because the polynomial is huge, so instead we work in $\mathbb{Z}_N[\zeta] / (m_0 - \zeta^2)$.

Here are some values from an example transcript (where we just send back x0):

In [1]:
from Crypto.Util.number import *

N = 971595721854019828628615615017444441126972049861721919804650748956780330711738883850893578496682360269992967877297127697576058755312276200741875803573585991285233284914491706368343846715439256237081908869230865886958788762329285313718959672196276247129196169037451863998490464128507918694684507097909017184901228089232855846158466183201817999895969739099922731532019488375643507692950137182165343058081768739955140783288068932955389241444220341756630754111528329
x0 = 876109414445200693942780122447130198715617327245210003691718863445602069564038517389140064990864490313532871436265756866577297330290644620759373445897899109765043898283242788124662572864645397818592934943785928195400967184172895983230251337985109919606230175001954620146468979251636282771470284029057994940471937771655239423755477099852763589929784346263099076428682549669842607587918541106052870503282533914729197176554161582155483992266361658934610957703589148
x1 = 712321106453975854512652491547525466487271019491273300532577053537724850554205292048273361401834198361730235234956813911090630909966738940613900563386286320947100905041625714455558032948347625309808563780797262612746301973277687638880039596205898270235641811723908807820120540811525335419889173755832076716185884183430786556464372234088599178082527318180639956987583997221466629065247246566926008673433524752640248649791902562674549163834804612907409768453183496
m0 = 966067897445086055323642101923671598460354632650794483522630815223818601568570497944402223617647616582408832040857035667771666164282409272456338507767829820702276544818042988245438238015087436447476394856681565057402417320329459390630385584457341109189878150755547004449504560599138538935544133634207211102817202159675235827724033194637094214582464480844450800788327288085325961630402818701590142642140139922338128947193962330589711527333590494313632560763723110
m1 = 792484298791895375357066277627896606715658344680802332117370125746676859329744661875320854410359563505564059187356678155849936643633694643239743757171092028582609384226256960434515536155911949053950533816012765836629011931025429520360460163735283698340408946697030517348647802182030216570497872288490810720886233190355937501068258910455228634821804855005231073493517553410398913661818138612695640624220492749333266300141851191214827436742889135903650162860049091

In [2]:
def polypow(base, exp, mod):
    return (mod.parent().quo(mod)(base) ** exp).lift()

𝜁 = polygen(Zmod(N))
p = gcd(N, ZZ((polypow(𝜁**3 - m1, N, 𝜁**2 - m0) + x0 - x1).monic()[0]**2 - m0))
print(f'{p = }')

p = 12896187564800986762862251971451401219338727671508894300548648924589067807077017591221851522407600622594870216875179484977119090323607432254215175855518789


Once you have $p$, learning $q+rf$ is exactly the same thing but easier.

In [3]:
qrf = ZZ(-(polypow(𝜁**3 + p**3 - m1, N, 𝜁**2 + p**2 - m0) + x0 - x1).monic()[0])
print(f'{qrf = }')

qrf = 568067568609625416975158891917034285170783409692117565676850445946154160034533404104494231984237175972223891551360308646463043376861286970009924365018058775353701049911717183090197879653324102292504119888486112600995948825276065696023638758015663332752059649257538791080721522735447321683655082237588992038834630480279139402327053196595931539380674559707173364593921285424775952322346618825691672456871443447016576425106216992133381420136940890775011202460537712


Now, $r$ is 512 bits and $f$ is $< 1024$ bits, so there's a good chance $q+rf < N$. If not we'll just reconnect out of laziness.

Since we know the value of $qr$ as well as the high bits of $f$, this implies we have the high bits of $q$ and $r$.

In [4]:
qr = N//p
q_approx = bytes_to_long(b'SEKAI{' + bytes(122)) * qr // qrf
print(f'{q_approx = }')

q_approx = 7755181395463725425956936424559583015016824092249894917926708555739041739063547675191445619341710787240223306799152440971453244064356892139634959294994886


Finally, notice that $q(q+rf) \equiv q^2 \pmod{qr}$, so we have a quadratic for $q$ (as a factor of $qr$) with known high bits. This means Coppersmith!

In [5]:
y = Zmod(qr)['y'].gen()
q = ZZ((y*(y-qrf))(y=y+q_approx).small_roots(X = 2**464, epsilon=0.08)[0]) + q_approx
print(f'{q = }')

q = 7755181395463763942061907622805461824657539873154240297729403461129869527727320739557970242849571329639625757554400116677906519715815413457706164430802237


We have all the values now! Let's print the flag.

In [6]:
print(long_to_bytes(qrf * q // qr))

b'SEKAI{this_challenge_was_originally_called_oblivion_but_there_was_an_ACSC_crypto_also_called_that}\xaeK\xdfj\xe4\x18\xce\xff\xd1.\xff\xccZ\xfd\xc5b\xee\xac\xef\xa5:<\xc0\xe1FT\xab\xf6I\xab'
